In [ ]:
from materials_2050_api_client import API_Client
import os

# Set the environment variable from Nicodemos
os.environ['DEVELOPER_TOKEN']= ''

# Now you can access it using getenv
developer_token = os.getenv('DEVELOPER_TOKEN')

client = API_Client(base_api_url = "https://app.2050-materials.com/", developer_token = developer_token)
products = client.get_products()
print(products)
